In [2]:
import os
import numpy as np
import pandas as pd

def snp_intersection(path):
    bim = [x for x in os.listdir(path) if '.bim' in x and x[0] != '.']
    chr_list = []
    for i in bim:
        try:
            print("Starting {}".format(i))
            chr_list.append(np.genfromtxt(
                    os.path.join(data_dir, i), 
                    dtype=str, delimiter='\t', usecols=1))
            print("Finsihed {}".format(i))
        except (IOError, ValueError) as e:
	        print("{} did not load".format(i))    
    intersection = set(chr_list[0]).intersection(*chr_list)
    return list(intersection)

def snp_pvals(snp_inter, snp, num=100000):
    df = pd.DataFrame(snp, columns=['snpid'])
    df['snp_copy'] = snp_inter
    match = df.set_index('snpid', 1).loc[snp]
    match = match.dropna(axis=0)
    match['snpid'] = match.index.values
    match.index = [x for x in range(match.shape[0])]
    match_psorted = match.sort_values('p')
    match_psorted.index = [x for x in range(match_psorted.shape[0])]
    return match_psorted.ix[:(num - 1), :]



/om/user/ysa/fmri_env/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [ ]:
data_dir = '/data/petryshen/yoel/EUR/eur'
files = [x for x in os.listdir(data_dir) if (('bgn' in x) and ('.bim' in x) and (x[0] != '.'))]
onekfile = pd.read_csv('100k_snp_intersections.csv')

def read_bim(data):
    bim = pd.read_csv(data, sep='\t', header=None)
    bim.columns = ['chrom','snpid','gd','pp','a1','a2']
    return bim

def match(matcher, matchie):
    snpm = matchie.set_index('snpid',1).loc[matcher.snpid]
    snpm['snpid'] = snpm.index.values 
    snpm.index= [x for x in range(snpm.shape[0])]
    s = snpm[['chrom', 'snpid', 'gd', 'pp', 'a1', 'a2']]
    return s

for f in files:
    try:
        bimfile = read_bim(os.path.join(data_dir, f)) 
        outfile = match(onekfile, bimfile)
        print(np.any(pd.isnull(outfile)))
        outfile.to_csv('{}'.format(f), sep='\t', index=None, header=None)
    except IOError:
        print("{} gave an error".format(f))


In [1]:
def match_bim(path, snps_pvals):
    files = [
        x for x in os.listdir(data_dir) if \
        (('bgn' in x) and ('.bim' in x) and (x[0] != '.'))
    ]
    